# Trabalho Prático 1  #
#### André Freitas PG54707 ####
#### Bruna Macieira PG54406 ####

## Descrição e Abordagem ao Problema ##

O principal objetivo deste trabalho passa por criar um comunicação privada assíncrona em modo  “Lightweight Cryptography” entre um agente Emitter e um agente Receiver. Esta implementação em Python passa pelo uso dos módulos Cryptography, Ascon e Asyncio, cada um deles dedicado a uma determinada ação ocorrente na implementação. 

Usamos o pacote Ascon para autenticar o criptograma envolvido na counicação bem como os metadados envolventes. O Ascon é uma família de algoritmos de encriptação autenticada e de hashing concebidos para serem leves e fáceis de implementar, mesmo com contramedidas adicionais contra ataques de canais laterais. Foi concebido por uma equipa de criptógrafos da Universidade de Tecnologia de Graz, da Infineon Technologies e da Universidade de Radboud: Christoph Dobraunig, Maria Eichlseder, Florian Mendel e Martin Schläffer.

O NIST anunciou a seleção da família Ascon como o padrão de criptografia leve em fevereiro de 2023, após receber feedback público em um workshop. O NIST está trabalhando com a equipe Ascon para elaborar os padrões de criptografia leve.

Asyncio é uma biblioteca para escrever código concorrente usando a sintaxe async/await, por isso tiramos proveito dessa funcionalidade para poder implementar a comunicação cliente-servidor.

O pacote Cryptography é uma biblioteca Python que fornece fórmulas criptográficas e primitivas para programadores. O uso deste pacote serve para implementar uma AEAD com “Tweakable Block Ciphers”. 

AEAD (Authenticated Encryption with Associated Data) é um esquema de criptografia que simultaneamente assegura a confidencialidade dos dados (também conhecida como privacidade: a mensagem criptografada é impossível de entender sem o conhecimento de uma chave secreta) e a autenticidade (ou seja, é inalterável: a mensagem criptografada inclui uma etiqueta de autenticação que o remetente só pode calcular possuindo a chave secreta).

##### Imports ####

In [1]:
from sage.all import *
from cryptography import *
from cryptography.fernet import Fernet
from asyncio import *

In [2]:
# Define a function
def quadratic_formula(a, b, c):
    discriminant = b^2 - 4*a*c
    if discriminant > 0:
        x1 = (-b + sqrt(discriminant))/(2*a)
        x2 = (-b - sqrt(discriminant))/(2*a)
        return x1, x2
    elif discriminant == 0:
        x = -b/(2*a)
        return x
    else:
        return "No real roots"

# Test the function
a = 1
b = -3
c = 2
roots = quadratic_formula(a, b, c)
print("Roots:", roots)


Roots: (2, 1)
